In [2]:
obj_opt=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
obj_misic=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
obj_misic_tighter=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
obj_misic_tighter_cuts=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
obj_misic_tighter_cuts_2=np.ones((len(n_feature_set),len(trees_to_test),n_rep))


for j,n_feature_trunc in enumerate(n_feature_set):
    for i,tree_size in enumerate(trees_to_test):
        for k in range(n_rep):
            x,y,feature_upper_bounds,feature_lower_bounds,n_data,n_feature=choose_dataset(dataset,n_data,n_feature_trunc)
            
            print('j',j)            
            print('i',i)
            print('k',k)
            
            if dataset!='abs_synth_noisy':
                x=x.iloc[:, 0:n_feature_trunc]
                feature_upper_bounds[0:n_feature_trunc]
                feature_lower_bounds[0:n_feature_trunc]
            n_feature=n_feature_trunc
            
            print("fitting tree")
            clf = RandomForestRegressor(n_estimators=tree_size,max_depth=20)
            #clf = RandomForestClassifier(n_estimators=tree_size,max_depth=12)
            clf.fit(x, y)
            print("tree fit \n")

            num_leaves=[]
            depth=[]
            for tree_index,tree_in_forest in enumerate(clf.estimators_):

                tree=tree_in_forest.tree_
                #pprint(getmembers(tree))
                #print()
                num_leaves.append(np.sum(tree.children_left== (-1)))
                depth.append(tree.max_depth)

            total_leaves=np.sum(num_leaves)
            total_depth=np.sum(depth)

            print("total_leaves",total_leaves)
            print("total_depth",total_depth)
            
            # optimal solution
            obj_opt[j,i,k],proj_leaf,runtime=misic_opt(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,False)
            # misic
            obj_misic[j,i,k],leaf_sol,runtime=misic_opt(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,make_relax)
            # elbow+expset
            obj_misic_tighter_cuts[j,i,k],leaf_sol=misic_opt_tighter_with_cuts(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,make_relax)
            # expset
            obj_misic_tighter[j,i,k],leaf_sol=misic_opt_tighter(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,make_relax)
            # elbow
            obj_misic_tighter_cuts_2[j,i,k],leaf_sol,run_time=misic_opt_tighter_with_cuts_2(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,make_relax)
            

n_feature_df=np.ones((n_rep,len(n_feature_set),len(trees_to_test)))
n_tree_size=np.ones((n_rep,len(n_feature_set),len(trees_to_test)))
for j,n_feature in enumerate(n_feature_set):
    for k,tree_size in enumerate(trees_to_test):
        for i in range(n_rep):
            n_feature_df[i,j,k]=n_feature
            n_tree_size[i,j,k]=tree_size
        
all_df=pd.DataFrame({'obj_misic_tighter': obj_misic_tighter.flatten(),
     'obj_misic': obj_misic.flatten(),'opt_obj': obj_opt.flatten(),
     'obj_misic_tighter_cuts': obj_misic_tighter_cuts.flatten(),
    'obj_misic_tighter_cuts_2': obj_misic_tighter_cuts_2.flatten(),
     'n_features':n_feature_df.flatten(),'tree_size':n_tree_size.flatten()})

all_df.to_csv("output/"+str(dataset)+"_relaxed_tree_size_w_cuts.csv")



for k,tree_size in enumerate(trees_to_test):
    
    df_temp=all_df[all_df.tree_size==tree_size]
    d = { 'expset': (df_temp.obj_misic_tighter-df_temp.opt_obj)/df_temp.opt_obj,
         'misic': (df_temp.obj_misic-df_temp.opt_obj)/df_temp.opt_obj,
         'elbow+expset': (df_temp.obj_misic_tighter_cuts-df_temp.opt_obj)/df_temp.opt_obj,
         'elbow': (df_temp.obj_misic_tighter_cuts_2-df_temp.opt_obj)/df_temp.opt_obj, 
         'n_features': df_temp.n_features}
    df = pd.DataFrame(data=d)

    means=df.groupby(['n_features'],as_index=False).mean()
    stds=df.groupby(['n_features'],as_index=False).std()/np.sqrt(10)
    temp=pd.melt(means, id_vars =['n_features'])
    temp['std_dev']=pd.melt(stds, id_vars =['n_features']).value

    fig = line(
        data_frame = temp,
        x = 'n_features',
        y = 'value',
        error_y = 'std_dev',
        error_y_mode = 'band',
        color = 'variable',
        title = '',
        markers = '.',
    )
    
    yaxis_title="Y Axis Title",
    fig.update_xaxes(type="log")
    fig.update_layout(yaxis_title="Optimality gap",width=500, height=500)
    fig.update_layout(legend={'title_text':''})
    fig.update_layout(yaxis_range=[0,np.max(temp.value)+np.max(temp.std_dev)])
    fig.update_layout(legend=dict(orientation = "h", yanchor="top",y=1.15))
    fig.show()
    fig.write_image("plots/"+str(dataset)+"_relaxed_tree_size"+str(tree_size)+".pdf")
    
    



NameError: name 'np' is not defined